In [67]:
import os
import pandas as pd

In [68]:
staging_folder = r'C:\Users\JenishaE\Documents\finalDE\finalProject-Data\staging'
prep_folder = r'C:\Users\JenishaE\Documents\finalDE\finalProject-Data\prep'

In [69]:
if not os.path.exists(prep_folder):
    os.makedirs(prep_folder)

In [70]:
file_path=os.path.join(staging_folder,'loaded_employee_skills_data.csv')

In [71]:
df_employee_skill=pd.read_csv(file_path)

In [72]:
df_employee_skill.head()

,emp_id,emp_name,skill_id,skill_name,proficiency_level,no_of_projects_done_in_that_skill,certifications
0,E747,Kelly Meza,S013,Financial Modeling,3.0,4.0,No
1,E1625,Shawn Rocha,S006,Python,5.0,2.0,Yes
2,E4373,Christine Choi,S002,React,7.0,6.0,Yes
3,E565,Dr. Chelsey Wallace,S012,Project Management,7.0,9.0,No
4,E3652,Gary Mendoza,S004,Docker,9.0,8.0,No


In [73]:
df_employee_skill.columns

Index(['emp_id', 'emp_name', 'skill_id', 'skill_name', 'proficiency_level',
       'no_of_projects_done_in_that_skill', 'certifications'],
      dtype='object')

In [74]:
df_employee_skill.shape

(5700, 7)

In [75]:
df_employee_skill.isnull().sum()

emp_id                                0
emp_name                              0
skill_id                              0
skill_name                            0
proficiency_level                    27
no_of_projects_done_in_that_skill    13
certifications                       25
dtype: int64

In [76]:
df_employee_skill.dtypes

emp_id                                object
emp_name                              object
skill_id                              object
skill_name                            object
proficiency_level                    float64
no_of_projects_done_in_that_skill    float64
certifications                        object
dtype: object

In [77]:
df_employee_skill['emp_id'] = df_employee_skill['emp_id'].astype('string')
df_employee_skill['emp_name'] = df_employee_skill['emp_name'].astype('string')
df_employee_skill['skill_id'] = df_employee_skill['skill_id'].astype('string')
df_employee_skill['skill_name'] = df_employee_skill['skill_name'].astype('string')
df_employee_skill['proficiency_level'] = df_employee_skill['proficiency_level'].astype('Int64')
df_employee_skill['no_of_projects_done_in_that_skill'] = df_employee_skill['no_of_projects_done_in_that_skill'].astype('Int64')
df_employee_skill['certifications'] = df_employee_skill['certifications'].astype('string')


In [78]:
print(df_employee_skill['proficiency_level'].min())
print(df_employee_skill['proficiency_level'].max())
print(df_employee_skill['no_of_projects_done_in_that_skill'].min())
print(df_employee_skill['no_of_projects_done_in_that_skill'].max())

1
10
1
12


In [79]:
def calculate_proficiency(no_of_projects):
    if pd.isna(no_of_projects):
        return None
    if no_of_projects > 5:
        return min(10, no_of_projects)  # Proficiency capped at 10
    else:
        return max(1, no_of_projects) 

In [80]:
df_employee_skill['proficiency_level'] = df_employee_skill.apply(
    lambda row: calculate_proficiency(row['no_of_projects_done_in_that_skill']) 
    if pd.isna(row['proficiency_level']) else row['proficiency_level'], 
    axis=1
)


In [81]:
df_employee_skill.isnull().sum()

emp_id                                0
emp_name                              0
skill_id                              0
skill_name                            0
proficiency_level                     0
no_of_projects_done_in_that_skill    13
certifications                       25
dtype: int64

In [82]:
def fill_projects_based_on_proficiency(row):
    if pd.isna(row['no_of_projects_done_in_that_skill']):
        if row['proficiency_level'] > 5:
            return 6 + (row['proficiency_level'] - 5)  # Projects in range of 6 to 12
        else:
            return row['proficiency_level']  # Projects below 6
    return row['no_of_projects_done_in_that_skill']  # If not NaN, keep the original value

In [83]:
df_employee_skill['no_of_projects_done_in_that_skill'] = df_employee_skill.apply(fill_projects_based_on_proficiency, axis=1)

In [84]:
df_employee_skill['certifications']=df_employee_skill['certifications'].fillna(df_employee_skill['proficiency_level'].apply(lambda x: 'Yes' if x > 5 else 'No'))

In [85]:
df_employee_skill.dtypes

emp_id                               string[python]
emp_name                             string[python]
skill_id                             string[python]
skill_name                           string[python]
proficiency_level                             int64
no_of_projects_done_in_that_skill             int64
certifications                       string[python]
dtype: object

In [86]:
df_employee_skill = df_employee_skill.drop_duplicates(subset=['emp_id', 'skill_id'])

In [87]:
df_employee_skill.shape

(5509, 7)

In [88]:
target_path=os.path.join(prep_folder,'employee_skill_prep.csv')
df_employee_skill.to_csv(target_path, index=False)